In [ ]:
!wget https://zenodo.org/record/841984/files/wili-2018.zip

--2019-12-23 15:03:43--  https://zenodo.org/record/841984/files/wili-2018.zip
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62403646 (60M) [application/octet-stream]
Saving to: ‘wili-2018.zip’

wili-2018.zip       100%[===================>]  59.51M  11.1MB/s    in 6.0s    

2019-12-23 15:03:50 (9.90 MB/s) - ‘wili-2018.zip’ saved [62403646/62403646]



In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

KeyboardInterrupt: ignored

In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [ ]:
!unzip wili-2018.zip


Archive:  wili-2018.zip
  inflating: x_train.txt             
  inflating: y_train.txt             
  inflating: x_test.txt              
  inflating: y_test.txt              
  inflating: labels.csv              
  inflating: README.txt              
  inflating: urls.txt                


In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import pickle

Using TensorFlow backend.


In [ ]:
print(tf.__version__)

1.15.0


In [ ]:
with open('x_train.txt') as f:
  data = f.read()

data_x_train = data.split('\n')
#print(len(data_x_train))           117501

with open('x_test.txt') as f:
  data = f.read()

data_x_test = data.split('\n')
#print(len(data_x_test))           117501

with open('y_train.txt') as f:
  data = f.read()

data_y_train = data.split('\n')
#print(len(data_y_train))           117501

with open('y_test.txt') as f:
  data = f.read()

data_y_test = data.split('\n')
#print(len(data_y_test))           117501

In [ ]:
data_x_train.pop(-1)
data_x_test.pop(-1)
data_y_train.pop(-1)
data_y_test.pop(-1)

''

In [ ]:
print(len(data_x_train))
print(len(data_x_test))
print(len(data_y_train))
print(len(data_y_test))

117500
117500
117500
117500


In [ ]:
data_x_train  = pd.DataFrame(data_x_train,columns=['sentence'])
data_y_train  = pd.DataFrame(data_y_train,columns=['language'])

data_x_test   = pd.DataFrame(data_x_test,columns=['sentence'])
data_y_test   = pd.DataFrame(data_y_test,columns=['language'])


In [ ]:

def process_sentence(sentence):
    '''Removes all special characters from sentence. It will also strip out
    extra whitespace and makes the string lowercase.
    '''
    return re.sub(r'[\\\\/:*«`\'?¿";!<>,.|()-_)(}{#$%@^&~+-=–—‘’“”„†•…′ⁿ№−、《》「」『』（），－：；]', '', sentence.lower().strip())

In [ ]:
x_train = data_x_train['sentence'].apply(process_sentence)
y_train = data_y_train['language']

x_test = data_x_test['sentence'].apply(process_sentence)
y_test = data_y_test['language']

In [ ]:
vectorizer = TfidfVectorizer(analyzer = 'char', norm = 'l2' , min_df=100, use_idf = True)
x_train = vectorizer.fit_transform(x_train)
x_test  = vectorizer.transform(x_test)

In [ ]:
print(x_train.shape)

(117500, 2420)


In [ ]:
print(x_test.shape)

(117500, 2420)


In [ ]:
languages = set(y_train)


In [ ]:
def create_lookup_table(text):
    """Create lookup tables for vocabulary
    :param text: The text split into words
    :return: A tuple of dicts (text_to_int, int_to_text)
    """
    
    text_to_int = { word : i for i, word in enumerate(text)}
    int_to_text = {   v  : k for k, v in text_to_int.items()}
    
    return text_to_int, int_to_text

In [ ]:
languages_to_int, int_to_languages = create_lookup_table(languages)

In [ ]:
def encode_to_int(data, data_to_int):
    """Converts all our text to integers
    :param data: The text to be converted
    :return: All sentences in ints
    """
    encoded_items = []
    for language in data: 
        if language in data_to_int:
          encoded_items.append([data_to_int[language]])
    return encoded_items

In [ ]:
y_train_encoded = OneHotEncoder().fit_transform(encode_to_int(y_train, languages_to_int)).toarray()
y_test_encoded = OneHotEncoder().fit_transform(encode_to_int(y_test, languages_to_int)).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [ ]:
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tf.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)

In [ ]:

model = Sequential()
model.add(Dense(512, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(len(languages), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/language_tf_idf_1.hdf5', monitor = "val_loss", verbose=1, save_best_only=True, mode = 'auto')

print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               1239552   
_________________________________________________________________
dense_1 (Dense)              (None, 235)               120555    
Total params: 1,360,107
Trainable params: 1,360,107
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Interrupt the training after 21 epoch otherwise it will overfit (because performane on valid set is not much increasing after 12 epoch)
score = model.fit(x_train, y_train_encoded, epochs=50, batch_size=32, validation_data=(x_test, y_test_encoded ), callbacks=[checkpointer])
    

Train on 117500 samples, validate on 117500 samples
Epoch 1/50
117472/117500 [============================>.] - ETA: 0s - loss: 0.5546 - acc: 0.8624
Epoch 00001: val_loss improved from 0.61427 to 0.52274, saving model to /content/drive/My Drive/language_tf_idf_1.hdf5
117500/117500 [==============================] - 25s 214us/sample - loss: 0.5546 - acc: 0.8624 - val_loss: 0.5227 - val_acc: 0.8683
Epoch 2/50
117408/117500 [============================>.] - ETA: 0s - loss: 0.4821 - acc: 0.8770
Epoch 00002: val_loss improved from 0.52274 to 0.48526, saving model to /content/drive/My Drive/language_tf_idf_1.hdf5
117500/117500 [==============================] - 26s 222us/sample - loss: 0.4820 - acc: 0.8771 - val_loss: 0.4853 - val_acc: 0.8784
Epoch 3/50
117472/117500 [============================>.] - ETA: 0s - loss: 0.4452 - acc: 0.8853
Epoch 00003: val_loss improved from 0.48526 to 0.46980, saving model to /content/drive/My Drive/language_tf_idf_1.hdf5
117500/117500 [=====================

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/My Drive/language_model_tf_idf_1.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
evaluate = model.evaluate(x_test, y_test_encoded, verbose=1)
print("Accuracy: %.2f%%" % (evaluate[1]*100))

117500/117500 [==============================] - 8s 70us/sample - loss: 0.4454 - acc: 0.8942
Accuracy: 89.42%


In [ ]:

predictions = model.predict(x_test)
 
pred_list = []
  
actual_list = []
 
for i in predictions:
  pred_list.append(np.argmax(i))
 
for i in y_test_encoded:
  actual_list.append(np.argmax(i))

y_actu = pd.Series(actual_list, name='Actual')
y_pred = pd.Series(pred_list, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
print(df_confusion)

Predicted  0    1    2    3    4    5    6    ...  228  229  230  231  232  233  234
Actual                                        ...                                   
0          469    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
1            0  334    0    0    0    0    0  ...    0    0    0    0    0    0    0
2            0    0  491    1    0    0    0  ...    0    0    0    0    0    0    0
3            0    0    0  413    0    0    0  ...    1    0    0    5    0    0    0
4            0    0    0    0  428    0    0  ...    0    0    0    1    0    0    0
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
230          0    0    0    0    0    0    0  ...    0    0  275    0    0    0    0
231          0    0    1    0    0    0    0  ...    0    0    0  423    0    0    0
232          0    0    0    0    0    0    0  ...    0    0    0    0  446    1    0
233          0    0    0    0    0    0    0  ...    0    0    0 

In [ ]:
def predict_sentence(sentence):
    """Converts the text and sends it to the model for classification
    :param sentence: The text to predict
    :return: string - The language of the sentence
    """
    
    # Clean the sentence
    sentence = process_sentence(sentence)
    
    # Transform and pad it before using the model to predict
    sentence = [sentence]
    x = vectorizer.transform(sentence)
        
    prediction = model.predict(x)
    # Get the highest prediction
    lang_index = np.argmax(prediction)

    return int_to_languages[lang_index]

In [ ]:
predict_sentence('''Ne l fin de l seclo XIX l Japon era inda çconhecido i sótico pa l mundo oucidental. 
Cula antroduçon de la stética japonesa, particularmente na Sposiçon Ounibersal de 1900, an Paris, l Oucidente 
adquiriu un apetite ansaciable pul Japon i Heiarn se tornou mundialmente coincido pula perfundidade, ouriginalidade i 
sinceridade de ls sous cuntos. An sous radadeiros anhos, alguns críticos, cumo George Orwell, acusórun Heiarn de trasferir
 sou nacionalismo i fazer l Japon parecer mais sótico, mas, cumo loufereciu al Oucidente alguns de sous purmeiros lampeijos
  de l Japon pré-andustrial i de l Período Meiji, sou trabalho inda ye balioso até hoije.''')

'mwl'

In [ ]:
predict_sentence("""बांग्लादेश के मुख्य न्यायाधीश का पद, बांग्लादेश सर्वोच्च न्यायिक पद है। इस पद पर विराजमान होने वाले पहले पदाधिकारी न्यायमूर्ति 
अब सादात मोहम्मह खान सयम थे, जोकि १६ दिसंबर १९७२ से नवंबर १९७५ तक इस पद पर रहे थे। तत्पश्चात, जनवरी २०१५ तक इस पद पर कुल २१ लोग विराजमान हो चुके हैं। 
वर्तमान मुख्य न्यायाधीश सुरेन्द्र कुमार सिन्हा इस पद पर १७ जनवरी २०१५ विराजमान हैं। वे हिन्दू धर्म के अनुयायी हैं, तथा बिष्णुप्रिय मणिपुरी समुदाय से आते हैं, तथा वे बांग्लादेश में
 किसी भी अल्पसंख्यक जातीय समूहों से नियुक्त पहली मुख्य न्यायाधीश है। न्यायमूर्ति भावनी प्रसाद सिन्हा भी एक ही समुदाय से हैं। न्यायमूर्ति महौदय नाज़मन आरा सुल्ताना इस पद को 
 सुशोभित करने वाली पहली महिला न्यायाधीश थीं, और मैडम जस्टिस कृष्णा देबनाथ बांग्लादेश की पहली महिला हिंदू न्यायाधीश है। वर्तमान में सुप्रीम कोर्ट में छह महिला न्यायाधीशों रहे हैं।""")

'hin'

In [ ]:
confusion_matrix(actual_list, pred_list)[7][7]

493

In [ ]:
int_to_languages[14]

'nob'

In [ ]:
predict_sentence('''Thou art the ruler of the minds of all people,
Dispenser of India's destiny.
Thy name rouses the hearts of Punjab, Sindh,
 Gujarat and Maratha,
Of the Dravida and Odisha
and Bengal''')

'eng'

In [ ]:
# Save and store vectorizer and int_to_languages to use them in other files by loading thwm

with open('/content/drive/My Drive/vectorizer.pk', 'wb') as fin:
  pickle.dump(vectorizer, fin, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/My Drive/int_to_languages.pk', 'wb') as fin:
  pickle.dump(int_to_languages, fin, protocol=pickle.HIGHEST_PROTOCOL)